In [31]:
import pickle
import random

import scipy
import numpy as np
import pandas as pd

try:
    import SparseSC as SC
except ImportError:
    raise RuntimeError("SparseSC is not installed. Use 'pip install -e .' or 'conda develop .' from repo root to install in dev mode")

In [32]:
random.seed(12345)
np.random.seed(101101001)

In [33]:
pkl_file = "../replication/smoking_fits.pkl"

In [34]:
smoking_df = pd.read_stata("../replication/smoking.dta")
smoking_df['year'] = smoking_df['year'].astype('int')
smoking_df = smoking_df.set_index(['state', 'year']).sort_index()
smoking_df.head()

cigsale  lnincome  beer  age15to24   retprice
state   year                                                  
Alabama 1970   89.800003       NaN   NaN   0.178862  39.599998
        1971   95.400002       NaN   NaN   0.179928  42.700001
        1972  101.099998  9.498476   NaN   0.180994  42.299999
        1973  102.900002  9.550107   NaN   0.182060  42.099998
        1974  108.199997  9.537163   NaN   0.183126  43.099998

In [35]:
Y = smoking_df[['cigsale']].unstack('year')
Y_cols = Y.columns
Y.head()

cigsale                                                  \
year               1970        1971        1972        1973        1974   
state                                                                     
Alabama       89.800003   95.400002  101.099998  102.900002  108.199997   
Arkansas     100.300003  104.099998  103.900002  108.000000  109.699997   
California   123.000000  121.000000  123.500000  124.400002  126.699997   
Colorado     124.800003  125.500000  134.300003  137.899994  132.800003   
Connecticut  120.000000  117.599998  110.800003  109.300003  112.400002   

                                                                         ...  \
year               1975        1976        1977        1978        1979  ...   
state                                                                    ...   
Alabama      111.699997  116.199997  117.099998  123.000000  121.400002  ...   
Arkansas     114.800003  119.099998  122.599998  127.300003  126.500000  ...   
California   127.099998  128.000000  126.400002  126.099998  121.900002  ...   
Colorado     131.000000  134.199997  132.000000  129.199997  131.500000  ...   
Connecticut  110.199997  113.400002  117.300003  117.500000  117.400002  ...   

                                                                         \
year               1991        1992        1993        1994        1995   
state                                                                     
Alabama      107.900002  109.099998  108.500000  107.099998  102.599998   
Arkansas     116.800003  126.000000  113.800003  108.800003  113.000000   
California    68.699997   67.500000   63.400002   58.599998   56.400002   
Colorado      90.199997   88.300003   88.599998   89.099998   85.400002   
Connecticut   86.699997   83.500000   79.099998   76.599998   79.300003   

                                                                        
year               1996        1997        1998        1999       2000  
state                                                                   
Alabama      101.400002  104.900002  106.199997  100.699997  96.199997  
Arkansas     110.699997  108.699997  109.500000  104.800003  99.400002  
California    54.500000   53.799999   52.299999   47.200001  41.599998  
Colorado      83.099998   81.300003   81.199997   79.599998  73.000000  
Connecticut   76.000000   75.900002   75.500000   73.400002  71.400002  

[5 rows x 31 columns]

In [36]:
T0 = 19
i_t = 2 #unit 3, but zero-index
treated_units = [i_t]
control_units = [u for u in range(Y.shape[0]) if u not in treated_units]
print(Y.shape)
print(control_units)

(39, 31)
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]


In [37]:
Y_names = Y.columns.get_level_values('year')
Y_pre_names = ["cigsale(" + str(i) + ")" for i in Y_names[:T0]]
print(Y.isnull().sum().sum()) #0
Y = Y.values
T = Y.shape[1]
T1 = T-T0
Y_pre,Y_post = Y[:,:T0], Y[:,T0:]

0


In [38]:
# Stata: synth cigsale beer(1984(1)1988) lnincome retprice age15to24 cigsale(1988) cigsale(1980) cigsale(1975), xperiod(1980(1)1988)  trunit(3) trperiod(1989) 

year_ind = smoking_df.index.get_level_values('year')
beer_pre = smoking_df.loc[np.logical_and(year_ind>=1984, year_ind<=1988),["beer"]]
Xother_pre = smoking_df.loc[np.logical_and(year_ind>=1980, year_ind<=1988), ['lnincome', 'retprice', 'age15to24']]
X_avgs = pd.concat((beer_pre.groupby('state').mean(), 
                    Xother_pre.groupby('state').mean())
                   , axis=1)

In [39]:
#X_spot = pd.DataFrame({'cigsale_75':smoking_df.xs(1975, level='year')["cigsale"], 
#                       'cigsale_80':smoking_df.xs(1980, level='year')["cigsale"], 
#                       'cigsale_88':smoking_df.xs(1988, level='year')["cigsale"]})
#X_orig = pd.concat((X_avgs, X_spot), axis=1)
#X_orig.isnull().sum().sum() #0

In [40]:
X_full = pd.concat((X_avgs, beer_pre.unstack('year'), Xother_pre.unstack('year')), axis=1)
X_full_names = [c[0] + "(" + str(c[1]) + ")" if len(c)==2 else c for c in X_full.columns]
X_full.isnull().sum().sum() #0
X_full = X_full.values
X_Y_pre = np.concatenate((X_full, Y_pre), axis=1)
X_Y_pre_names = X_full_names + Y_pre_names
X_Y_pre_names_arr = np.array(X_Y_pre_names)

In [41]:
def print_summary(full_fit, Y_pre, Y_post, Y_sc, show_noNH = True):
    full_Y_pre_sc,full_Y_post_sc = Y_sc[:,:T0], Y_sc[:,T0:]
    print("V: " + str(np.diag(full_fit.V)))
    print("V>0: " + str(np.diag(full_fit.V)[np.diag(full_fit.V)>0]))
    print("#V>0: " + str(sum(np.diag(full_fit.V>0))))
    full_Y_pre_effect_c = Y_pre[control_units, :] - full_Y_pre_sc[control_units, :]
    full_Y_post_effect_c = Y_post[control_units, :] - full_Y_post_sc[control_units, :]
    
    print(X_Y_pre_names_arr[np.diag(full_fit.V)>0])

    def print_seg_info(arr, seg_name)
        print("Avg bias " + seg_name + ": " + str(arr.mean()))
        print(scipy.stats.ttest_1samp(arr.flatten(), popmean=0)) 
        print("Avg MSE " + seg_name + ": " + str(np.mean(np.power(arr, 2))) )
        print("Avg max abs val " + seg_name + ":" + str(np.mean(np.amax(np.abs(arr), axis=0))))
    
    print_seg_info(full_Y_pre_effect_c, "pre")
    print_seg_info(full_Y_post_effect_c, "post")

    NH_idx = 20 #1-based index including treatment is 22
    if show_noNH:    
        full_Y_pre_effect_c_noNH = np.delete(full_Y_pre_effect_c, NH_idx, axis=0)
        full_Y_post_effect_c_noNH = np.delete(full_Y_post_effect_c, NH_idx, axis=0)    
        
        print_seg_info(full_Y_pre_effect_c_noNH, "pre (no-NH)")
        print_seg_info(full_Y_post_effect_c_noNH, "post (no-NH)")


Fast

In [12]:
if False:
    fast_fit = SC.fit_fast(X_Y_pre, Y_post, treated_units=[i_t])
    #print(len(np.diag(fast_fit.V)))
    #print(np.diag(fast_fit.V))
    #Y_post_sc = fast_fit.predict(Y_post)
    #Y_pre_sc = fast_fit.predict(Y_pre)
    #post_mse = np.mean(np.power(Y_post[control_units, :] - Y_post_sc[control_units, :], 2))
    #pre_mse = np.mean(np.power(Y_pre[control_units, :] - Y_pre_sc[control_units, :], 2))
    #print(pre_mse) #192.210632448
    #print(post_mse) #129.190437803
    #print(X_Y_pre_names_arr[fast_fit.match_space_desc>0])

Full

In [45]:
#use_est_fn = True
#if use_est_fn:
#    est_fit = SC.estimate_effects(
#        outcomes=Y,
#        unit_treatment_periods=X,
#        covariates=None,
#        fast = False,
#        cf_folds = len(control_units), #sync with helper
#        **kwargs
#    )
#    full_fit = 
#else: 
print("Start time: {}".format(datetime.datetime.now().replace(microsecond=0)))
full_fit = SC.fit(X_Y_pre, Y_post, treated_units=[i_t], verbose=0, progress=False, print_path=False)
print("End time: {}".format(datetime.datetime.now().replace(microsecond=0)))


Done!


In [13]:
full_Y_sc = full_fit.predict(Y)
print_summary(full_fit, Y_pre, Y_post, full_Y_sc)

V: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.62581428e-06 1.67298271e-04
 1.25447705e-04 2.83218864e-05 1.20684653e-04 2.90538206e-05
 1.68471635e-04 1.91157505e-04 1.95644283e-04 2.87258604e-04
 4.25739528e-04 4.62630652e-04 6.40482407e-04]
V>0: [5.62581428e-06 1.67298271e-04 1.25447705e-04 2.83218864e-05
 1.20684653e-04 2.90538206e-05 1.68471635e-04 1.91157505e-04
 1.95644283e-04 2.87258604e-04

In [14]:
honest_predictions, cf_fits = SC.get_c_predictions_honest(X_Y_pre[control_units,:], Y_post[control_units,:], Y[control_units,:], 
                                                 w_pen=full_fit.initial_w_pen, v_pen=full_fit.initial_v_pen, cf_folds=38, verbose=1, progress=False, fast=False)

 |>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [21]:
full_Y_sc[control_units,:] = honest_predictions
print_summary(full_fit, Y_pre, Y_post, full_Y_sc)

V: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.62581428e-06 1.67298271e-04
 1.25447705e-04 2.83218864e-05 1.20684653e-04 2.90538206e-05
 1.68471635e-04 1.91157505e-04 1.95644283e-04 2.87258604e-04
 4.25739528e-04 4.62630652e-04 6.40482407e-04]
V>0: [5.62581428e-06 1.67298271e-04 1.25447705e-04 2.83218864e-05
 1.20684653e-04 2.90538206e-05 1.68471635e-04 1.91157505e-04
 1.95644283e-04 2.87258604e-04

In [20]:
full_Y_sc.shape

(39, 31)

In [ ]:
honest_predictions_df = pd.DataFrame(honest_predictions, columns=Y_cols, index=control_units).stack(level="year")
honest_predictions_df.to_stata("smoking_sparsesc.dta")

# Full - flat
Since we don't fit v, we don't have to do out-of-sample refitting

In [15]:
full_fit_flat = SC._fit_fast_inner(X_Y_pre, X_Y_pre, Y_post, V=np.repeat(1,X_Y_pre.shape[1]), treated_units=[i_t])

In [16]:
full_flat_Y_sc = full_fit_flat.predict(Y)
print_summary(full_fit_flat, Y_pre, Y_post, full_flat_Y_sc)

V: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
V>0: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
#V>0: 55
Avg bias pre: 0.003513363039932098
Ttest_1sampResult(statistic=0.05033751319674089, pvalue=0.959867371950055)
Avg MSE pre: 3.5123625231089566
Avg effect post: -0.40206830732880827
Ttest_1sampResult(statistic=-0.7260495769207599, pvalue=0.46818168129036175)
Avg MSE post: 139.69517126549485
['beer' 'lnincome' 'retprice' 'age15to24' 'beer(1984)' 'beer(1985)'
 'beer(1986)' 'beer(1987)' 'beer(1988)' 'lnincome(1980)' 'lnincome(1981)'
 'lnincome(1982)' 'lnincome(1983)' 'lnincome(1984)' 'lnincome(1985)'
 'lnincome(1986)' 'lnincome(1987)' 'lnincome(1988)' 'retprice(1980)'
 'retprice(1981)' 'retprice(1982)' 'retprice(1983)' 'retprice(1984)'
 'retprice(1985)' 'retprice(1986)' 'retprice(1987)' 'retprice(1988)'
 'age15to24(1980)' 'age15to24(1981)' 'age15to24(1

write-out

In [22]:
#Write out
with open(pkl_file, "wb" ) as output_file:
    pickle.dump( (full_fit),  output_file) #full_fit_flat

In [11]:
#Read back
with open(pkl_file, "rb" ) as input_file:
    (full_fit) = pickle.load(input_file)